<a href="https://colab.research.google.com/github/cda79/IAT360-LLM/blob/main/IAT360_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Required libraries

In [1]:
!pip install transformers torch datasets

In [2]:
!pip install transformers torch accelerate

##Import dataset

In [8]:
from datasets import load_dataset
dataset = load_dataset("lanretto/shakespeare-vs-modern-dialogue")
dataset

DatasetDict({
    train: Dataset({
        features: ['speaker', 'line', 'label'],
        num_rows: 325001
    })
    validation: Dataset({
        features: ['speaker', 'line', 'label'],
        num_rows: 40625
    })
    test: Dataset({
        features: ['speaker', 'line', 'label'],
        num_rows: 40626
    })
})

In [7]:
## For now, unused
# from datasets import load_dataset
# dataset = load_dataset("madha98/Shakespeare")
# dataset

README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

shakespeare1.txt: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/124456 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 124456
    })
})

##Data preprocessing

In [9]:
#example print
dataset['train'][0]

{'speaker': 'Helena',
 'line': 'To die upon the hand I love so well.',
 'label': 1}

In [10]:
#shuffle and split the dataset into a smaller amount
#select only the first 100 and shuffle them up
dataset = dataset['train'].shuffle(seed=30).select(range(100))
dataset

Dataset({
    features: ['speaker', 'line', 'label'],
    num_rows: 100
})

In [11]:
# create test and train dataset from this shuffled amount
# 80-20 split
dataset = dataset.train_test_split(train_size=0.8, seed=40)
dataset


DatasetDict({
    train: Dataset({
        features: ['speaker', 'line', 'label'],
        num_rows: 80
    })
    test: Dataset({
        features: ['speaker', 'line', 'label'],
        num_rows: 20
    })
})